# MATH 405 - Numerical Methods for Differential Equations

# MATH 607 - Topics in Numerical Analysis

[[Instructor: Christoph Ortner]](http://www.math.ubc.ca/~ortner/)  [[CANVAS]](https://canvas.ubc.ca/courses/55324)

# WS - Implementation of Finite-Difference Schemes in 2D, 3D


We will restrict attention to the $d$-dimensional Laplace equation, 
$$\begin{aligned}
    -\Delta u = f, \qquad x \in \Omega, \\ 
        u = 0, & x \in \partial \Omega,
\end{aligned}$$
where $\Omega$ is a domain in $\mathbb{R}^d$ and $\partial\Omega$ denotes its boundary. 

Implementing finite-difference schemes for general domains is possible but requires a large number of "tricks", hence we will restrict ourselves only to tensor-product domains, e.g., the unit cube
$$
    \Omega := (0, 1)^d
$$
but other tensor-product domains are equally easy to treat.

The main idea is to exploit the tensor product structure of the Laplace operator, e.g., if $d = 2$, 
$$
    \Delta u = \frac{\partial^2 u}{\partial x_1^2}  + \frac{\partial^2 u}{\partial x_2^2} 
     = \Big( D^2 \otimes I + I \otimes D^2 \Big) u,
$$
where $D$ denotes the univariate derivative operator. Suppose now that $D^2_h$ denotes a univariate finite difference discretisation of $D^2$ then the resulting finite-difference discretisation of the laplace operator becomes 
$$
    \Delta_h u = \Big( D_h^2 \otimes I + I \otimes D_h^2 \Big) u.
$$


Let us consider the case we are most interested in, 
$$
    D_h^2 V_n = \frac{V_{n+1} - 2 V_n + V_{n-1}}{h^2}
$$
and assume that $U_{nm}$ is a 2-dimensional grid function, then 
$$\begin{aligned}
    \big(D_h^2 \otimes I\big) U_{nm} &= \frac{U_{n+1,m} - 2 U_{nm} + U_{n-1,m}}{h^2}, \\ 
    \big(I \otimes D_h^2\big) U_{nm} &= \frac{U_{n,m+1} - 2 U_{nm} + U_{n,m-1}}{h^2}, \\ 
\end{aligned}$$
and hence 
$$
    \Delta_h U_{nm} = \big(D_h^2 \otimes I + I \otimes D_h^2\big) U_{nm} = 
    \frac{U_{n+1,m} + U_{n,m+1} + U_{n-1,m} + U_{n,m-1}  - 4 U_{nm} }{h^2}.
$$
One can readily check exactly as in the univariate case that, if $u_{nm} = u(nh, mh)$ then 
$$
    \Delta_h u_{nm} = \Delta u_{nm} + O(h^2).
$$

The generalisation to $d$ dimensions is 
$$
    \Delta_hU_{{\bf n}} = \sum_{i = 1}^d \frac{U_{{\bf n} + {\bf e}_i} - 2 U_{{\bf n}} + U_{{\bf n} - {\bf e}_i}}{h^2}
    = \Big( D_h^2 \otimes I \otimes \cdots \otimes I
            + I \otimes D_h^2 \otimes I \otimes \cdots \otimes I
            + \cdots
            + I \otimes \cdots \otimes I \otimes D_h^2 \Big) U_{{\bf n}}.
$$

In [ ]:
include("../math405.jl")

In [ ]:
# this implementation uses dirichlet boundary conditions only.
L1d(N) = Tridiagonal(-N^2 * ones(N-2), 2*N^2*ones(N-1), -N^2*ones(N-2))
Id(N) = sparse(I, (N-1,N-1))
L2d(N) = sparse(kron(L1d(N), Id(N)) + kron(Id(N), L1d(N)))

In [ ]:
display(L1d(5))

In [ ]:
display(Matrix(L2d(5)))

In [ ]:
N = 20 
# solve the laplace equation - Δu = 1
L = L2d(20)
F = ones(size(L,1))
U = L \ F;

In [ ]:
# postprocess for plotting; use plotly backend to manipulate the plot
Plots.plotly()
U0 = zeros(N+1, N+2) 
U0[2:N, 2:N] = reshape(U, (N-1, N-1))
x = range(0, 1, length = N+1)
surface(x, x, U0)

The convergence theory / error analysis for the $d$-dimensional setting is essentially equivalant to the one-dimensional one. I.e. we can prove that 

$$
    \| U - u \|_\infty \leq C h^2.
$$

Let's check this numerically via the method of manufactured solutions: If we take $u(x_1, x_2) = \sin(\pi x_1) \sin(\pi x_2)$, then we have $- \Delta u = \pi^2 u$.

In [ ]:
ufun = (x1, x2) -> sin(pi*x1)*sin(pi*x2)
err = []
NN = (2).^(3:7)
for N in NN 
    x = range(0, 1, length=N+1)
    o = ones(N+1)
    u = ufun.(x*o', o*x')
    F = 2*pi^2 * u[2:N,2:N][:]
    U = L2d(N) \ F 
    push!(err, norm(u[2:N,2:N][:] - U, Inf))
end 

In [ ]:
plot(NN.^(-1), err, lw=3, ms = 5, m = :o, label = "error", xscale = :log10, yscale = :log10, size = (400, 200))
plot!(NN[3:end].^(-1), NN[3:end].^(-2), lw=2, ls=:dash, c=:black, label = "h^2", legend = :topleft)